In [1]:
import pandas as pd
import csv

In [ ]:
tabla_series = pd.read_csv("./components/info_gustos_series.csv", encoding="utf-8")
tabla_movies = pd.read_csv("./components/info_gustos_peliculas.csv", encoding="utf-8")
tabla_series.columns = ["Nombre", "Carrera", "Edad", "Series", "Genre"]
tabla_movies.columns = ["Nombre", "Carrera", "Edad", "Pelis", "Genre"]
# print(tabla_series["Series"])
# print(tabla_movies["Pelis"])
peliculas = tabla_movies["Pelis"]
series = tabla_series["Series"]
# print(peliculas)

In [3]:
# Displaying final output format
from IPython.display import display, Markdown, Latex
# LangChain Dependencies
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
# from langchain_community.chat_models import ChatOllama
from langchain_ollama.llms import OllamaLLM
from langgraph.graph import START, END, StateGraph
# For State Graph 
from typing_extensions import TypedDict
import json
import os

In [4]:
# Defining LLM
local_llm = 'llama3'
llama3 = OllamaLLM(model=local_llm, temperature=0)
llama3_json = OllamaLLM(model=local_llm, format='json', temperature=0)

In [5]:
# Generation Prompt

media_analizer = PromptTemplate(
    template="""
    
    <|begin_of_text|>
    
    <|start_header_id|>system<|end_header_id|> 
    
    You are an expert in all tv shows and movies, you will be given an array of may tv shows, and your job is to return
    a your response with the format JSON: {{"NameOfTheShow": "genre"}} with only one genre for each show.
    Do not give any further explanation on your response.
    Try using general purpose genres, for example, sitcom and comedy are the same.
    
    <|eot_id|>
    
    <|start_header_id|>user<|end_header_id|>
    
    Shows: {shows} 
    
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["shows"],
)

series_chain = media_analizer | llama3_json | JsonOutputParser()


In [6]:
# Node - Generate

def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    
    # print("Step: Generating Final Response")
    shows = state["shows"]
    # print(shows, movies)

    # Answer Generation
    generation = series_chain.invoke({"shows": shows})

    return {"generation": generation}

In [7]:
# Graph State
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        search_query: revised question for web search
        context: web_search result
    """
    shows: str
    generation : str

In [8]:
# Build the nodes
workflow = StateGraph(GraphState)
workflow.add_edge(START, "generate")
workflow.add_node("generate", generate)
workflow.add_edge("generate", END)

# Compile the workflow
local_agent = workflow.compile()


In [9]:
def run_agent_series():
    shows = []
    for serie in series:
        output = local_agent.invoke({"shows": serie})
        # print("=======")
        print(output["generation"])
        shows.append(output["generation"])
    return shows
def run_agent_movies():
    movies = []
    for movie in peliculas:
        output = local_agent.invoke({"shows": movie})
        # print("=======")
        print(output["generation"])
        movies.append(output["generation"])

    return movies    

In [10]:
all_shows = run_agent_series()

{'Gilmore Girls': 'Drama'}
{'Gossip Girl': 'Drama'}
{'The Vampire Diaries': 'Drama'}
{'Gilmore Girls': 'Drama'}
{'Riverdale': 'Drama'}
{'Control-Z': 'Thriller'}
{'Inside Job': 'Documentary'}
{'Gossip Girl': 'Drama'}
{'Friends': 'Comedy'}
{'Virgin River': 'Drama'}
{'Casas de papel': 'Thriller'}
{'South Park': 'Comedy'}
{'Breaking Bad': 'Crime'}
{'Supernatural': 'Fantasy'}
{'La Rosa de Guadalupe': 'Soap Opera'}
{'Yellowstone': 'Western'}
{'Club-de-Cuervos': 'Comedy'}
{'Suits': 'Drama'}
{'La Rosa de Guadalupe': 'Soap Opera'}
{'Suits': 'Drama'}
{'El Señor de los Cielos': 'Drama'}
{'Arcane': 'Animation'}
{'Brooklyn Nine-Nine': 'Comedy'}
{'The Boys': 'Action'}
{'Invincible': 'Superhero'}
{'Enchanted': 'Fantasy'}
{'La Casa De Papel': 'Crime'}
{'Breaking Bad': 'Crime'}
{'Modern Family': 'Comedy'}
{'Ginny & Georgia': 'Drama'}
{'Love Alarm': 'Romance'}
{'Emily Dickinson': 'Biography'}
{'Bridgerton': 'Romance'}
{'The Glory': 'Drama'}
{'Alice in the Borderlands': 'Mystery'}
{'Rick and Morty': 'Sci

In [ ]:
all_movies = run_agent_movies()
# print(peliculas)

{'Pirates of the Caribbean': 'Adventure'}
{'Finding Nemo': 'Animation'}
{'La Ila Siniestra': 'Mystery'}
{'Un-pequeño-Favor': 'Comedy'}
{'Interstellar': 'Science Fiction'}
{'Gone Girl': 'Thriller'}
{'Teléfono-Negro': 'Thriller'}
{'The Submarine': 'Drama'}
{'Clueless': 'Comedy'}
{"Something's Gotta Give": 'Romantic Comedy'}
{'Princess Diaries 2': 'Comedy'}
{'Juego de Gemelas': 'Drama'}
{'Interstellar': 'Science Fiction'}
{'Coraline': 'Fantasy'}
{'Insidious': 'Horror'}
{'Dune': 'Science Fiction'}
{'Nosferatu': 'Horror'}
{'The Neon Demon': 'Horror'}
{'Coraline': 'Fantasy'}
{'Los Juegos del Hambre en Llamas': 'Drama'}
{'Black Swan': 'Drama'}
{'Fantasia': 'Musical'}
{'Twilight': 'Fantasy'}
{'Yo-Yo-Boing': 'Comedy'}
{'Interstellar': 'Science Fiction'}
{'Avatar': 'Science Fiction'}
{'Shadowhunters': 'Fantasy'}
{'Interstellar': 'Science Fiction'}
{'Inception': 'Action'}
{'La-La-Land': 'Romance'}
{'Oppenheimer': 'Biography'}
{'El Efecto Mariposa': 'Drama'}
{'La-La Land': 'Musical'}
{'Coraline': 

In [ ]:
df = pd.DataFrame([{"Show": k, "Genre": v} for s in all_shows for k, v in s.items()])
print(df)
df.to_csv("dict_series.csv", index=False)
df = pd.DataFrame([{"Movie": k, "Genre": v} for s in all_movies for k, v in s.items()])
print(df)
df.to_csv("dict_movies.csv", index=False)
# i = 0
# with open('dict.csv', 'w') as csv_file:  
#   writer = csv.writer(csv_file)
#   for o in output:
#     for key, value in o.items():
#       print(key, value)
#       if (key != None and value != None):
#         writer.writerow([key, value])

                             Show    Genre
0                   Gilmore Girls    Drama
1                     Gossip Girl    Drama
2             The Vampire Diaries    Drama
3                   Gilmore Girls    Drama
4                       Riverdale    Drama
..                            ...      ...
106                    The Office   Comedy
107                Anne with an E    Drama
108                          Dark  Mystery
109               Shadow and Bone  Fantasy
110  Problema-de-los-tres-cuerpos  Mystery

[111 rows x 2 columns]
                        Movie            Genre
0    Pirates of the Caribbean        Adventure
1                Finding Nemo        Animation
2            La Ila Siniestra          Mystery
3            Un-pequeño-Favor           Comedy
4                Interstellar  Science Fiction
..                        ...              ...
100              Interstellar  Science Fiction
101               Untouchable            Crime
102                   Rocky-4        

In [ ]:
#References:
# Pandas: https://pandas.pydata.org/docs/getting_started/index.html
# Langchain: https://langchain-ai.github.io/langgraph/tutorials/workflows/#prompt-chaining 